**Company XYZ is a very early stage startup. They allow people to stream music from their mobile for free. Right now, they still only have songs from the Beatles in their music collection, but they are planning to expand soon. They still have all their data in json ﬁles and they are interested in getting some basic info about their users as well as building a very preliminary song recommendation model in order to increase user engagement. Working with json ﬁles is important. If you join a very early stage start-up, they might not have a nice database and all data will be in jsons. Third party data are often stored in json ﬁles as well.**

##### goal: increase user engagement

The company CEO asked you very speciﬁc questions:

1) What are the top 3 and the bottom 3 states in terms of number of users? 

2) What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically deﬁne user engagement. 

3) What the CEO cares about here is in which states users are using the product a lot/very little. The CEO wants to send a gift to the ﬁrst user who signed-up for each state. That is, the ﬁrst user who signed-up from California, from Oregon, etc. Can you give him a list of those users? 

4) Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model. 

5) How would you set up a test to check whether your model works well and is improving engagement?

In [1]:
import json

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")

In [2]:
with open('song.json') as f:
    data = pd.DataFrame(json.load(f))
    
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 4000 non-null   object
 1   user_id            4000 non-null   int64 
 2   user_state         4000 non-null   object
 3   user_sign_up_date  4000 non-null   object
 4   song_played        4000 non-null   object
 5   time_played        4000 non-null   object
dtypes: int64(1), object(5)
memory usage: 187.6+ KB


In [4]:
display(data.describe())
display(data.describe(include='object'))

,user_id
count,4000.000000
mean,101.574000
std,58.766835
min,1.000000
25%,48.000000
50%,102.000000
75%,155.000000
max,200.000000


,id,user_state,user_sign_up_date,song_played,time_played
count,4000,4000,4000,4000,4000
unique,4000,41,20,97,3997
top,UQYFXAYBBE,New York,2015-05-19,Come Together,2015-06-15 13:23:02
freq,1,469,532,408,2


In [5]:
# convert datetime variables to the correct data type
data['user_sign_up_date'] = pd.to_datetime(data['user_sign_up_date'])
data['time_played'] = pd.to_datetime(data['time_played'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 4000 non-null   object        
 1   user_id            4000 non-null   int64         
 2   user_state         4000 non-null   object        
 3   user_sign_up_date  4000 non-null   datetime64[ns]
 4   song_played        4000 non-null   object        
 5   time_played        4000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 187.6+ KB


## Q1. What are the top 3 and the bottom 3 states in terms of number of users?


In [6]:
data['user_state'].value_counts().nlargest(3)

New York      469
California    425
Texas         230
Name: user_state, dtype: int64

In [7]:
data['user_state'].value_counts().nsmallest(3)

Kansas          8
Connecticut    16
New Mexico     17
Name: user_state, dtype: int64

## Q2. What are the top 3 and the bottom 3 states in terms of user engagement? 

**You can choose how to mathematically define user engagement.** 

**What the CEO cares about here is in which states users are using the product a lot/very little.**

In [8]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [9]:
# measure the engagement by average number of songs played per user in the state

In [10]:
engagement = data.groupby('user_state').count()['song_played'] / data.groupby('user_state')['user_id'].nunique()
print("Top 3 States in term of User Engagement: ", list(engagement.nlargest(3).index))
print("Bottom 3 States in term of User Engagement: ", list(engagement.nsmallest(3).index))

Top 3 States in term of User Engagement:  ['Nebraska', 'Alaska', 'Mississippi']
Bottom 3 States in term of User Engagement:  ['Kansas', 'Virginia', 'Minnesota']


### Q3. The list of first sign-up users for each state.

In [11]:
import pandasql as ps

In [12]:
query = "select distinct user_id, user_state from data where (user_id, user_sign_up_date) in (select user_id, min(user_sign_up_date) from data group by user_state)"

first_signups = ps.sqldf(query)

first_signups

,user_id,user_state
0,3,Ohio
1,6,New Jersey
2,174,Rhode Island
3,11,Pennsylvania
4,85,Missouri
5,18,Maryland
6,165,Idaho
7,34,Kentucky
8,127,Connecticut
9,32,Wisconsin


In [13]:
# specifically, if you only want to know the user_id of the first sign_ups:
print("List of First-Signups in Each State: ", list(first_signups['user_id']))

List of First-Signups in Each State:  [3, 6, 174, 11, 85, 18, 165, 34, 127, 32, 19, 23, 41, 60, 70, 45, 13, 15, 125, 5, 106, 173, 64, 20, 178, 102, 7, 2, 119, 135, 78, 4, 29, 1, 134, 105, 39, 177, 142, 8, 50]


### Q4. Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.

In [14]:
# to solve this problem, we basically need to build a recommender system model


# conner cases:
# 1. what if the song enetred is always the end song for each user

# consider building:
# 1. collaborative recommender system: based on the reaction of other users. 
#     Basically, we need the rating of the songs, yet we don't have here, 
#     so I decided to generate a variable evaluating the spong, which is the total number of time played by users.

In [15]:
df = data.copy()

In [16]:
# Bild the song-user matrix

song_user = df.groupby(['song_played','user_id'])['song_played'].count().unstack().fillna(0)
song_user = song_user.astype(int)

song_user.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,3,0,2,0,0,0,0,...,0,0,3,3,0,2,0,0,2,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Build Song-Song similarity matrix
from sklearn.preprocessing import normalize

# there're some frequent users/songs
# thus we need to normalize it otherwise they would dominate the similarity score
song_user_norm = normalize(song_user, axis=1)

# calculate the similarity matrix
similarity = np.dot(song_user_norm, song_user_norm.T)
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

similarity_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,Baby You're A Rich Man,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.246021,0.067651,0.108483,0.192538,0.341760,0.322896,0.138092,0.223237,0.175762,...,0.462712,0.055237,0.509397,0.225503,0.427027,0.033826,0.331593,0.339179,0.079727,0.0
A Hard Day's Night,0.246021,1.000000,0.000000,0.000000,0.100000,0.136931,0.111803,0.000000,0.000000,0.091287,...,0.259548,0.129099,0.210099,0.000000,0.112987,0.000000,0.050000,0.195468,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.067651,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.109435,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.108483,0.000000,0.000000,1.000000,0.000000,0.154303,0.094491,0.109109,0.000000,0.000000,...,0.116991,0.000000,0.138107,0.089087,0.190982,0.000000,0.000000,0.146845,0.000000,0.0
Across The Universe,0.192538,0.100000,0.000000,0.000000,1.000000,0.091287,0.000000,0.000000,0.000000,0.000000,...,0.138426,0.000000,0.116722,0.000000,0.075324,0.000000,0.000000,0.043437,0.000000,0.0


In [18]:
# define a function to get top 10 values from

def get_TopK(song, similarity, k=1):
    
    df = similarity.loc[song].sort_values(ascending=False)[1:k+1].reset_index()
    
    df = df.rename(columns={'song_played':"Recommended Song", song:"Similarity Score"})
    
    return df

In [19]:
get_TopK("Eight Days A Week", similarity_df, 3)

,Recommended Song,Similarity Score
0,Hey Jude,0.399035
1,Come Together,0.392338
2,Let It Be,0.388770


### Q5: How would you set up a test to check whether your model works well and is improving engagement?

Here, I would decide to go with an A/B Testing to test the improvement of engagement.

Metric: engagement per day

Hypothesis:
H0: User engagement is the same in the test group and the control group.
H1: User engagement is higher in the test group compared to the user engagement in the control group.

Statistics:
one-tailed t-test

Test Setup:
the model should be tested by market. So, I would match comparable markets in pairs, and within each pair, give everyone in one market song recommendations based on the new model (test group), and everyone in the other market song recommendations without the model (control). The "comparable" are based on the initial engagement of the users, for example, in one pair, the two markets should both be of high engagement.
The experiment should last for a while, say, based on the traffic of our app, the power/significance level we choose. Stop collecting data when the sample size meet the requirement, and run the on-tailed t-test to see if the difference between the two is significant or not. Repeat the experiment several times on  different pairs of the market to get a more sound result.